In [22]:
import sys
import itertools
from math import sqrt
from operator import add
from os.path import join, isfile, dirname
import math
from pyspark import SparkConf, SparkContext
from pyspark.mllib.recommendation import ALS
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.clustering import LDA, LDAModel
from sklearn.neighbors import NearestNeighbors
from pyspark.mllib.tree import RandomForest, RandomForestModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
import textwrap
from datetime import date, datetime
import pyproj
from math import cos, sin
import matplotlib.pyplot as plt
from pyspark.mllib.clustering import GaussianMixture, GaussianMixtureModel
import numpy as np
from pyspark.mllib.stat import MultivariateGaussian
from pyspark.mllib.linalg import DenseMatrix

In [23]:
#f = sc.textFile("file:///bgdt/yellow.2000000.csv") # file:// for local files (default hdfs)
f = sc.textFile("gs://dataproc-97276f09-d220-4be8-9ace-dae0bcad2c57-us/test/yellow.1605.csv") # file:// for local files (default hdfs)
# 7575962 credit card trips
#f = sc.textFile("file:///bgdt/yellow.50000.csv")
#f = sc.textFile("file:///bgdt/yellow.csv")
header = f.first()
[ str(i) + ' ' + x for i, x in enumerate(header.split(",")) ]

[u'0 VendorID',
 u'1 tpep_pickup_datetime',
 u'2 tpep_dropoff_datetime',
 u'3 passenger_count',
 u'4 trip_distance',
 u'5 pickup_longitude',
 u'6 pickup_latitude',
 u'7 RatecodeID',
 u'8 store_and_fwd_flag',
 u'9 dropoff_longitude',
 u'10 dropoff_latitude',
 u'11 payment_type',
 u'12 fare_amount',
 u'13 extra',
 u'14 mta_tax',
 u'15 tip_amount',
 u'16 tolls_amount',
 u'17 improvement_surcharge',
 u'18 total_amount']

In [25]:
# def hour2ts(h):
#     if h in {}:
        
#     elif

dtf = "%Y-%m-%d %H:%M:%S"  # date format

def defineProj():
    return( pyproj.Proj("+init=epsg:2908 +ellps=WGS84 +datum=WGS84 +proj=utm") )

# theta, xoffset, yoffset are decided by seeing figures
def rotateManhattanY(latm, lonm, theta=-0.496, yoffset=4239000):
    lonm, latm = defineProj()(lonm, latm)
    return( cos(theta)*latm - sin(theta)*lonm - yoffset )

def rotateManhattanX(latm, lonm, theta=-0.496, xoffset=-1635000):
    lonm, latm = defineProj()(lonm, latm)
    return( sin(theta)*latm + cos(theta)*lonm - xoffset )

def inUWS(x, y):
    return(  955 < x and x < 2283 and
            9443 < y and y < 15003 )

def inUpperTimesSquare(x, y):
    return( 1500 < x and x < 4350 and
            8000 < y and y < 9500 )

def in14smallStreets(x, y):
    return( 2900 < x and x < 3100 and
            8125 < y and y < 9225 )

# translation bounds From: http://www.spatialreference.org/ref/epsg/2908/

univ = f.filter(lambda x: x != header)\
.map(lambda l: l.split(","))\
.map(lambda c:
     {        'pt': c[ 1],
       'isWeekend': int(datetime.strptime(c[1],dtf).isoweekday()>5),
                    # isoweekday() returns Monday as 1, Sundey as 7
              'ph': datetime.strptime(c[1],dtf).hour//2,
              'dt': c[ 2],
             'len': (datetime.strptime(c[2],dtf) -
                     datetime.strptime(c[1],dtf)).seconds // 60,
                    # actual trip duration in minutes
             'dst': float(c[ 4]),
              'po': float(c[ 5]),
              'pa': float(c[ 6]),
              'do': float(c[ 9]),
              'da': float(c[10]),
               '$': c[11], # payment type: 1 is credit card (tip recorded)
            'fare': float(c[12]),
              'ex': float(c[13]),
             'tip': float(c[15]),
            'toll': float(c[16]),
      })\
.filter(lambda x: x['$'] == '1' )\
.filter(lambda x: x['len'] >= 2 and x['len'] <= 120 )\
.filter(lambda x:  40.4700 < x['pa'] and x['pa'] <  41.3100 and
                  -74.2700 < x['po'] and x['po'] < -71.7500 and
                   x['po'] != 0 and x['pa'] != 0 )\
.map(lambda c:
     { 'isWeekend': c['isWeekend'],
              'ph': c['ph'],
             'len': c['len'],
      'isHighHpay': int((c['tip']*60/c['len'])>=12),
             'dst': c['dst'],
              'po': c['po'],
              'pa': c['pa'],
              'px': rotateManhattanX(c['pa'], c['po']),
              'py': rotateManhattanY(c['pa'], c['po']),
              'do': c['do'],
              'da': c['da'],
            'fare': c['fare'],
              'ex': c['ex'],
            'toll': c['toll']
      })\
.cache()

# can insert before .cache()

# .filter(lambda x: inUWS(x['px'], x['py']) )\
# .filter(lambda x: inUpperTimesSquare(x['px'], x['py']) )\

univ.take(1)

[{'da': 40.730098724365234,
  'do': -73.9839859008789,
  'dst': 3.6,
  'ex': 0.5,
  'fare': 15.0,
  'isHighHpay': 0,
  'isWeekend': 1,
  'len': 17,
  'pa': 40.76803970336914,
  'ph': 0,
  'po': -73.98590087890625,
  'px': 1991.9836131976917,
  'py': 9282.009793342091,
  'toll': 0.0}]

In [29]:
#exemode = sys.argv[1]
exemode = 'rf'
if exemode == 'rf': # random forest
    print("random forest")
    master = univ.filter(lambda x: inUpperTimesSquare(x['px'], x['py']) )
    #nrCards = univ.count()
    print("%d credit card trips" % (nrCards) )
else:
    print("mixture gaussian model")
    master = univ.filter(lambda x: in14smallStreets(x['px'], x['py']) )
    print("  number of rows: " + str(master.count()))


random forest
7575962 credit card trips


In [30]:
#preped = univ.map(lambda l: (l['dst'], l['tip']) )

if exemode == 'rf':
    moment = 'finished'
    if moment == 'before': 
        print("use before")
        preped = master.map(lambda l:
                          LabeledPoint(l['isHighHpay'],
                                       [l['isWeekend'], l['po'], l['pa'], l['ph'], l['ex']]))
    elif moment == 'during':
        print("use during")
        preped = master.map(lambda l:
                          LabeledPoint(l['isHighHpay'],
                                       [l['isWeekend'], l['po'], l['pa'], l['ph'], l['ex'],
                                        l['dst'], l['toll']
                                       ]))
    else:
        print("use finished")
        preped = master.map(lambda l:
                          LabeledPoint(l['isHighHpay'],
                                       [l['isWeekend'], l['po'], l['pa'], l['ph'], l['ex'],
                                        l['dst'], l['toll'],
                                        l['fare'], l['len'], l['do'], l['da']
                                       ]))
    preped.take(1)
    
    trRDD, vaRDD, teRDD = preped.randomSplit([6,2,2], seed=0L)
    print("  training RDD:", trRDD.take(1))
    print("validation RDD:", vaRDD.map(lambda x: x.features ).take(1))

use finished
('  training RDD:', [LabeledPoint(0.0, [1.0,-73.9792938232,40.7557640076,0.0,0.5,0.63,0.0,5.0,5.0,-73.9880142212,40.7584686279])])
('validation RDD:', [DenseVector([1.0, -73.9898, 40.7622, 0.0, 0.5, 0.6, 0.0, 4.5, 3.0, -73.9793, 40.7621])])


In [31]:
if exemode != 'rf':
    pickup_locationRDD = master.map(lambda l: Vectors.dense(l['py']))
    print(pickup_locationRDD.take(1))

In [32]:
initModel = GaussianMixture.train(pickup_locationRDD, k=14, convergenceTol=1e+5, seed=1234)
initModel

NameError: name 'pickup_locationRDD' is not defined

In [114]:
initModel.gaussians

[MultivariateGaussian(mu=DenseVector([8808.8344]), sigma=DenseMatrix(1, 1, [61848.7152], 0)),
 MultivariateGaussian(mu=DenseVector([8706.1932]), sigma=DenseMatrix(1, 1, [92596.5652], 0)),
 MultivariateGaussian(mu=DenseVector([8649.9059]), sigma=DenseMatrix(1, 1, [101729.2174], 0)),
 MultivariateGaussian(mu=DenseVector([8848.3886]), sigma=DenseMatrix(1, 1, [69281.7783], 0)),
 MultivariateGaussian(mu=DenseVector([8630.0653]), sigma=DenseMatrix(1, 1, [102299.5305], 0)),
 MultivariateGaussian(mu=DenseVector([8824.2428]), sigma=DenseMatrix(1, 1, [67967.339], 0)),
 MultivariateGaussian(mu=DenseVector([8636.7486]), sigma=DenseMatrix(1, 1, [97175.0077], 0)),
 MultivariateGaussian(mu=DenseVector([8322.6781]), sigma=DenseMatrix(1, 1, [25504.2029], 0)),
 MultivariateGaussian(mu=DenseVector([8512.7717]), sigma=DenseMatrix(1, 1, [78264.9321], 0)),
 MultivariateGaussian(mu=DenseVector([8980.0908]), sigma=DenseMatrix(1, 1, [27235.833], 0)),
 MultivariateGaussian(mu=DenseVector([8562.7125]), sigma=Den

In [96]:
for i in range(1,len(initModel.gaussians)):
    initModel.gaussians[i].sigma.values = int(initModel.gaussians[0].sigma.values)
#initModel.gaussians[1].sigma[3] = initModel.gaussians[1].sigma._replace(DenseMatrix(1, 1, [300], 0))
#initModel.gaussians[1].sigma
GaussianMixture.

In [119]:
initModel.gaussians[1] = initModel.gaussians[1]._replace(MultivariateGaussian(mu=1, sigma=1))
initModel.gaussians[1]

TypeError: _replace() takes exactly 1 argument (2 given)

In [108]:
aaa = DenseMatrix(1,1,[1],0)
print(aaa)
aaa.values = [2]
aaa
initModel.gaussians[0].sigma.values = np.array([2])
initModel.gaussians[0].sigma.values
#np.array([2])

DenseMatrix([[ 1.]])



array([ 61848.71515456])

In [33]:
if exemode == 'rf':
    model = RandomForest.trainClassifier(trRDD, numClasses=2,
                                         categoricalFeaturesInfo = {},
                                 numTrees=5, featureSubsetStrategy="sqrt",
                                impurity="gini", maxDepth=4, maxBins=32, seed = 1)
    # gini is faster
    model.save(sc, "myRandomForestModel_old")
    print("model saved")
    preds = model.predict(vaRDD.map(lambda x: x.features ))
    labelsAndPredictions = vaRDD.map(lambda lp: lp.label).zip(preds)
    testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(vaRDD.count())
    print('Test Accuracy = %f percent' % (100-round(testErr,7)*100))
    # print('Learned classification forest model:')
    # print(model.toDebugString()) # messy
    
    # Note: for unfiltered data,
    # 69 -before-> 72% -finish-> 75%
else:
    # we model 14 streets between 42st and 57st
    model = GaussianMixture.train(pickup_locationRDD, k=14, convergenceTol=1e-1, seed=1234)
    

model saved
Test Accuracy = 76.668180 percent


In [34]:
if exemode == 'rf':
    nActuNotHigh = labelsAndPredictions.filter(lambda lp: lp[0] == 0 ).count()
    nActuHigh    = labelsAndPredictions.filter(lambda lp: lp[0] == 1 ).count()
    nPredNotHigh = labelsAndPredictions.filter(lambda lp: lp[1] == 0 ).count()
    nPredHigh    = labelsAndPredictions.filter(lambda lp: lp[1] == 1 ).count()

    print("Baseline : %f     " % (float(max(nActuNotHigh,nActuHigh)) / float(nActuNotHigh+nActuHigh)))
    print("Actual  0: %9d    1: %9d     " % (nActuNotHigh, nActuHigh) )
    print("Predict 0: %9d    1: %9d     " % (nPredNotHigh, nPredHigh) )
else:
    print(model.gaussians)

Baseline : 0.515122     
Actual  0:    151301    1:    142418     
Predict 0:    168997    1:    124722     


In [256]:

# def rotateManhattanY(latm, lonm, theta=-0.496, yoffset=4239000):
#     # theta, xoffset, yoffset are decided by seeing figures
#     return([(cos(theta)*la - sin(theta)*lo - yoffset) for la,lo in zip(latm, lonm) ])
# def rotateManhattanX(latm, lonm, theta=-0.496, xoffset=-1635000):
#     return([(sin(theta)*la + cos(theta)*lo - xoffset)  for la,lo in zip(latm, lonm) ]) 

# MAYBE-LATER: currently it works but needs more explicit settings.
# 
# p = pyproj.Proj("+init=epsg:2908 +ellps=WGS84 +datum=WGS84 +proj=utm")



# poms, pams = p([ x['po']   for x in dots[1:40000] ],
#                [ x['pa']   for x in dots[1:40000] ])
# print("pom, pam = (%f, %f)" % (poms[0], pams[0]) )

# px = rotateManhattanX(pams, poms, theta)
# py = rotateManhattanY(pams, poms, theta)

# print("px,py = (%f, %f)" % (px[0], py[0]) )

# plt.plot(dots[1:1000]['px'],dots[1:1000]['py'], 'ro', markersize=.5 )
# plt.xlim(xmin=0, xmax=6000)
# plt.ylim(ymin=0, ymax=20000)

# plt.show()

In [269]:
# Upper West Side
print("Teachers College - 121st Y: %5f" % rotateManhattanY(latm= 40.811176, lonm = -73.961526) )
print("The Northern End - 125st Y: %5f" % rotateManhattanY(latm= 40.815378, lonm = -73.958365) )
print("The Southern End - 59st Y: %5f" %  rotateManhattanY(latm= 40.771913, lonm = -73.991137) )
print("The Western End (Bay): %5f" %      rotateManhattanX(latm= 40.772487, lonm = -73.996697) )
print("The Eastern End - Cent. Park %5f"% rotateManhattanX(latm= 40.769585, lonm = -73.980830) )




Teachers College - 121st Y: 14466.660687
The Northern End - 125st Y: 15003.933584
The Southern End - 59st Y: 9443.014457
The Western End (Bay): 955.688724
The Eastern End - Cent. Park 2282.630162


In [51]:
print("Times Square %5f"% rotateManhattanY(latm= 40.758591, lonm = -73.984795) )
print("42st Bryant Pk%5f" % rotateManhattanY(latm= 40.754822, lonm = -73.984122) )
       

Times Square 8410.672915
42st Bryant Pk8072.623159
